1. Load MongoDB Snapshot into pandas

In [60]:
import pandas as pd

df = pd.read_json("snapshots/immobilien_snapshot.json")
print(df.shape)
df.head(3)

(444, 36)


,_id,url,abstellraum,address,ausstattung,balkon,barrierefrei,bautyp,befristung,böden,...,sonstiges,stockwerk,teilmöbliert_/_möbliert,terrasse,titel,verfügbar,wohnfläche,zimmer,zusatzinformationen,zustand
0,{'$oid': '6959375616a2bbfc2e12991f'},https://www.willhaben.at/iad/immobilien/d/miet...,NaN,"Zelda-Kaplan-Weg 6, 1100 Wien, 10. Bezirk, Fav...",,11 m²,1.0,Neubau,3 Jahre,Parkett,...,Diese Wohnung bietet urbanes Leben mit einziga...,1,NaN,None,Biotope City Wienerberg - naturnahes Wohnen in...,ab 22.12.2025,53 m²,2,,Sehr gut/gut
1,{'$oid': '6959375616a2bbfc2e129920'},https://www.willhaben.at/iad/immobilien/d/miet...,NaN,"Laxenburger Straße 2D, 1100 Wien, 10. Bezirk, ...",,"6,5 m²",1.0,Neubau,3 Jahre,Parkett,...,,4,1.0,None,"Smarte 1,5-Zimmer-Wohnung mit 6,5m2 Balkon im ...",ab 01.03.2026,36 m²,1,,Neuwertig
2,{'$oid': '6959375616a2bbfc2e129921'},https://www.willhaben.at/iad/immobilien/d/miet...,1.0,"Doktor-Adolf-Schärf-Platz, 1220 Wien, 22. Bezi...",Loggiaanzahl: 1\nAbstellraum Anzahl: 1\nLoggia...,None,NaN,Neubau,3 Jahr(e),Parkett,...,,12,NaN,None,Wunschlos Glücklich - Moderne 2-Zimmer-Wohnung...,ab sofort,None,2,Stockwerk: 12. Etage\nAnzahl Badezimmer: 1\nKl...,Neuwertig


2. Clean Data

In [61]:
import re
import numpy as np

df["preis_alt"] = df["preis"]
df["wohnfläche_alt"] = df["wohnfläche"]

# $oid -> _id
df["_id"] = df["_id"].apply(lambda x: x.get("$oid") if isinstance(x, dict) else x)

# preis: "€ 1.120" -> 1120.0
def parse_eur_price(x):
    if pd.isna(x):
        return np.nan
    s = str(x)
    s = re.sub(r"[^\d,\.]", "", s)
    s = s.replace(".", "").replace(",", ".")
    return float(s) if s else np.nan

df["preis"] = df["preis"].apply(parse_eur_price)
df["balkon"] = df["balkon"].apply(parse_eur_price)
df["terrasse"] = df["terrasse"].apply(parse_eur_price)

# wohnfläche: "53 m²" -> 53.0
def parse_m2(x):
    if pd.isna(x):
        return np.nan
    s = str(x)

    s = s.replace("m²", " ").strip()

    # bei doppelte Zahlen nur die erste Zahl nehmen
    m = re.search(r"\d{1,3}(?:[.\s]\d{3})*(?:,\d+)?|\d+(?:,\d+)?", s)
    if not m:
        return np.nan

    num = m.group(0)
    num = num.replace(" ", "")
    num = num.replace(".", "")
    num = num.replace(",", ".")

    try:
        return float(num)
    except ValueError:
        return np.nan

df["wohnfläche"] = df["wohnfläche"].apply(parse_m2)

# zimmer: "2" -> 2.0
df["zimmer"] = pd.to_numeric(
    df["zimmer"].astype(str).str.replace(",", ".", regex=False),
    errors="coerce"
)

# Bezirk: 10. Bezirk / 03. Bezirk etc. -> int
# (Postleitzahl 1100 -> 10, 1030 -> 3)
def extract_district(addr):
    if pd.isna(addr):
        return np.nan
    s = str(addr)

    # "03. Bezirk", "10. Bezirk", etc.
    m = re.search(r"(\d{1,2})\.\s*Bezirk", s)
    if m:
        d = int(m.group(1))
        return d if 1 <= d <= 23 else np.nan

    # Postleitzahl: 1010..1230 -> 1..23
    m = re.search(r"\b(10\d{2}|11\d{2}|12\d{2})\b", s)
    if m:
        plz = int(m.group(0))
        d = (plz % 100) // 10
        return d if 1 <= d <= 23 else np.nan

    return np.nan

df["Bezirk"] = df["address"].apply(extract_district).astype("float")

print(df[["preis", "wohnfläche", "zimmer", "Bezirk", "address"]].head(10))

    preis  wohnfläche  zimmer  Bezirk  \
0  1120.0       53.00     2.0    10.0   
1   890.0       36.00     1.0    10.0   
2  1300.0         NaN     2.0    22.0   
3   880.0       42.00     2.0    10.0   
4  1990.0       90.00     3.0     3.0   
5  1800.0       76.00     3.0     9.0   
6  1180.0       67.25     3.0    10.0   
7  2038.0      109.00     4.0    18.0   
8  1350.0       80.00     3.0    10.0   
9   930.0       42.69     2.0    16.0   

                                             address  
0  Zelda-Kaplan-Weg 6, 1100 Wien, 10. Bezirk, Fav...  
1  Laxenburger Straße 2D, 1100 Wien, 10. Bezirk, ...  
2  Doktor-Adolf-Schärf-Platz, 1220 Wien, 22. Bezi...  
3                   1100 Wien, 10. Bezirk, Favoriten  
4                  1030 Wien, 03. Bezirk, Landstraße  
5                  1090 Wien, 09. Bezirk, Alsergrund  
6                   1100 Wien, 10. Bezirk, Favoriten  
7                     1180 Wien, 18. Bezirk, Währing  
8  Sonnleithnergasse, 1100 Wien, 10. Bezirk, Favo... 

3. Data Split

In [63]:
df_model = df.copy()

df_model = df_model.dropna(subset=["preis", "wohnfläche", "zimmer", "Bezirk"])

# true/false columns
bool_cols = ["einbauküche", "fahrstuhl", "balkon", "terrasse", "garage", "parkplatz", "teilmöbliert_/_möbliert"]
for c in bool_cols:
    df_model[c] = df_model[c].fillna(False).astype(int)

cat_cols = ["bautyp", "zustand"]
for c in cat_cols:
    df_model[c] = df_model[c].fillna("Unknown").astype(str)


X = df_model[["wohnfläche", "zimmer", "Bezirk",
              "einbauküche", "fahrstuhl", "balkon", "terrasse",
              "garage", "parkplatz", "teilmöbliert_/_möbliert",
              "bautyp", "zustand"]]

y = df_model["preis"]

X = pd.get_dummies(X, columns=["bautyp", "zustand"])

In [64]:
from sklearn.model_selection import train_test_split

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y,
    test_size=0.15,
    random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval,
    test_size=0.15 / 0.85,
    random_state=42
)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)


Train: (275, 18) Val: (59, 18) Test: (60, 18)


4. Training

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Train
rf = RandomForestRegressor(
    n_estimators=600,
    random_state=42,
    n_jobs=-1,
    min_samples_leaf=2
)

rf.fit(X_train, y_train)

# Evaluate
def eval_split(name, Xp, yp):
    pred = rf.predict(Xp)
    mae = mean_absolute_error(yp, pred)
    rmse = np.sqrt(mean_squared_error(yp, pred))
    r2 = r2_score(yp, pred)
    print(f"{name}: MAE={mae:.0f} €, RMSE={rmse:.0f} €, R²={r2:.3f}")
    return pred

pred_train = eval_split("Train", X_train, y_train)
pred_val   = eval_split("Val  ", X_val, y_val)
pred_test  = eval_split("Test ", X_test, y_test)

test_err = pd.DataFrame({
    "actual": y_test.values,
    "pred": pred_test,
    "abs_err": np.abs(y_test.values - pred_test),
}, index=y_test.index).sort_values("abs_err", ascending=False)

test_err.head(10)

Train: MAE=237 €, RMSE=377 €, R²=0.839
Val  : MAE=305 €, RMSE=403 €, R²=0.751
Test : MAE=286 €, RMSE=406 €, R²=0.629


,actual,pred,abs_err
92,3150.68,1627.100434,1523.579566
89,2539.41,3752.376725,1212.966725
394,1577.92,2529.642415,951.722415
419,585.00,1357.115715,772.115715
36,1780.00,2494.186897,714.186897
379,1980.00,1314.562268,665.437732
181,2196.00,2816.468194,620.468194
88,368.00,944.000804,576.000804
184,543.40,1029.101558,485.701558
238,1230.00,756.873722,473.126278


In [97]:
def predict_rent(example_dict):
    row = pd.DataFrame([[0]*len(X_train.columns)], columns=X_train.columns)

    for k, v in example_dict.items():
        if k in row.columns:
            row.at[0, k] = v

    return float(rf.predict(row)[0])

example = {
    "wohnfläche": 52,
    "zimmer": 2,
    "Bezirk": 22,
    "einbauküche": 0,
    "fahrstuhl": 1,
    "balkon": 1,
    "terrasse": 0,
    "garage": 0,
    "parkplatz": 1,
    "teilmöbliert_/_möbliert": 0
}
predict_rent(example)


1133.8624461357429